In [1]:
git_repo = '../../..'

import sys
import os

#sys.path.insert(0, './../../../LinearResponseVariationalBayes.py')
sys.path.insert(0, os.path.join(git_repo, 'BNP_sensitivity/GMM_clustering/'))
#'./../')

# Linear response libraries
import LinearResponseVariationalBayes as vb
import LinearResponseVariationalBayes.SparseObjectives as obj_lib
#import LinearResponseVariationalBayes.ExponentialFamilies as ef

# My libraries
import gmm_clustering_lib as gmm_utils
import modeling_lib 
import functional_sensitivity_lib as fun_sens_lib 
import utils_lib

from scipy import spatial
import scipy.cluster.hierarchy as sch

import matplotlib.pyplot as plt
%matplotlib inline

import time
from tqdm import tqdm

from copy import deepcopy

# Just while you're experimenting
from autograd import numpy as np
from autograd import scipy as sp

# import numpy as np
# import scipy as sp

np.random.seed(453453)

import json 

In [2]:
# Load results from previous fit.

json_input_file = os.path.join(
    git_repo, 'BNP_sensitivity/GMM_clustering/iris_fits/ryan_iris_bnp_full_data_fit.json')

with open(json_input_file, 'r') as fp:
    fit_dict = json.load(fp)
    model = gmm_utils.get_model_from_checkpoint(fit_dict)
    model_ = gmm_utils.get_model_from_checkpoint(fit_dict)
    best_param = model.global_vb_params.get_free()
    kl_hessian = gmm_utils.get_kl_hessian_from_checkpoint(fit_dict)

In [3]:
print(model.global_vb_params.get_free().shape)
print(model.prior_params.get_free().shape)
print(model.weights.get_free().shape)

(190,)
(14,)
(150,)


In [5]:
moment_model = gmm_utils.InterestingMoments(model)
linear_sens = gmm_utils.LinearSensitivity(model, moment_model, kl_hessian)

Prior Hessian...
Data Hessian...
Linear systems...
Done.


In [6]:
prior_cross_hess = model.get_kl_prior_cross_hess(model.prior_params.get_free(), best_param)
print(np.linalg.norm(prior_cross_hess))

97.78613994340358


In [9]:
dgdeta = np.zeros(kl_hessian.shape[0])
dgdeta[0] = 1

gh = np.linalg.solve(kl_hessian, dgdeta)
print(gh.shape)

(190,)


In [10]:

q_class = fun_sens_lib.StickSensitivity(model, best_param, kl_hessian, dgdeta)
q_class.model.global_vb_params.set_free(best_param)
print(q_class.get_log_q_logit_stick(0.5, 0))
print(q_class.log_q_logit_stick_obj.fun_free(best_param, logit_v=0.5, k=0))

#print(best_param)
q_class.model.global_vb_params.set_free(best_param)
#print(q_class.log_q_logit_stick_obj.fun_free_grad(best_param, logit_v=0.5, k=0))

print('---------\n')
print(q_class.get_log_q_logit_stick(0.5, 0))
print(q_class.get_log_p0_logit_stick(0.1))

print('\n--------- all:')
print(q_class.get_log_q_logit_all_sticks(0.1))

print('---------\n')
print(q_class.get_single_stick_influence(logit_v=0, k=0))
print(q_class.get_all_stick_influence(logit_v=0.1))


NameError: name 'obj_lib' is not defined

In [ ]:
v_grid = np.linspace(1e-3, 1 - 1e-3, num=100)
logit_v_grid = np.log(v_grid / (1 - v_grid))

plt.figure()
influence_grid = np.array(
    [ q_class.get_all_stick_influence(logit_v=logit_v) for
      logit_v in logit_v_grid ])
plt.plot(v_grid, influence_grid, 'k')
plt.figure()
plt.plot(logit_v_grid, influence_grid, 'k')

logit_v_min = np.log(1e-3 / (1 - 1e-3))
logit_v_max = np.log((1 - 1e-3) / 1e-3)
logit_v_grid = np.linspace(logit_v_min, logit_v_max, num=100)
v_grid = np.exp(logit_v_grid) / (1 + np.exp(logit_v_grid))

influence_grid = np.array(
    [ q_class.get_all_stick_influence(logit_v=logit_v) for
      logit_v in logit_v_grid ])
plt.figure()
plt.plot(v_grid, influence_grid, 'k')
plt.figure()
plt.plot(logit_v_grid, influence_grid, 'k')



In [ ]:
v_grid = np.linspace(1e-3, 1 - 1e-3, num=100)
logit_v_grid = np.log(v_grid / (1 - v_grid))

for k in range(model.k_approx - 1):
    influence_grid = np.array(
        [ q_class.get_single_stick_influence(logit_v=logit_v, k=k) for
          logit_v in logit_v_grid ])
    plt.figure()
    plt.plot(logit_v_grid, influence_grid, 'k')
    plt.title(k)